# EasyVVUQ - Basic Concepts

This tutorial shows a simple EasyVVUQ workflow in action. The example used here is a simulation of a robotic arm. Description of it can be found [here](https://www.sfu.ca/~ssurjano/robot.html).

The usage of the application is:

```robot <input_file>```

It outputs a single file called output.json, which will look like ```{'distance': 30}``` or similar where ```distance``` is the distance between the plane to which the robot arm is attached and the end of the arm.

The robot.template is a template input file, in JSON format

```{"outfile": "$out_file", "theta1": $theta1, "theta2" $theta2, "theta3": $theta3, "theta4": $theta4, "L1": $L1, "L2": $L2, "L3": $L3, "L4": $L4}```

The thetas are angles (in radians) between the four robotic arm segments and L's are their lengths (in arbitrary units). The values for each key are tags (signified by the $ delimiter) which will be substituted by EasyVVUQ with values to sample the parameter space. In the following tutorial, the template will be used to generate files called input.json that will be the input to each run of ```robot```.

So, for example:

In [ ]:
!echo "{\"outfile\": \"output.json\", \"theta1\": 0.5, \"theta2\": 0.5, \"theta3\": 0.25, \"theta4\": 0.75, \"L1\": 1, \"L2\": 1, \"L3\": 1, \"L4\": 1}" > input.json

In [ ]:
!./robot input.json

In [ ]:
!cat output.json

# Campaign

In [1]:
import easyvvuq as uq
import chaospy as cp

In [2]:
campaign = uq.Campaign('robot')

In [3]:
params = {
    "theta1": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5}, 
    "theta2": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5}, 
    "theta3": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5}, 
    "theta4": {"type": "float", "min": 0.0, "max": 1.0, "default": 0.5},
    "L1": {"type": "float", "min": 0.0, "default": 1.0},
    "L2": {"type": "float", "min": 0.0, "default": 1.0},
    "L3": {"type": "float", "min": 0.0, "default": 1.0},
    "L4": {"type": "float", "min": 0.0, "default": 1.0},
    "outfile": {"type": "string", "default": "output.json"}
}

In [4]:
encoder = uq.encoders.GenericEncoder(template_fname='robot.template', delimiter='$', target_filename='input.json')
decoder = uq.decoders.JSONDecoder(target_filename='output.json', output_columns=['distance'])

In [5]:
campaign.add_app(name='robot', params=params, encoder=encoder, decoder=decoder)

In [6]:
vary = {
    "theta1": cp.Uniform(0, 1),
    "theta2": cp.Uniform(0, 1),
    "theta3": cp.Uniform(0, 1),
    "theta4": cp.Uniform(0, 1)
}

In [7]:
campaign.set_sampler(uq.sampling.PCESampler(vary=vary, polynomial_order=3))

In [8]:
execution = campaign.sample_and_apply(0, action=uq.actions.ExecuteLocalV2("robot input.json"), batch_size=8)

In [9]:
execution.start()

[<Future at 0x1275514c0 state=running>,
 <Future at 0x11126e520 state=running>,
 <Future at 0x12763fbb0 state=running>,
 <Future at 0x12763f970 state=running>,
 <Future at 0x12761b910 state=running>,
 <Future at 0x12761bee0 state=running>,
 <Future at 0x12761f430 state=running>,
 <Future at 0x12761f9d0 state=running>,
 <Future at 0x12761feb0 state=pending>,
 <Future at 0x1275fb430 state=pending>,
 <Future at 0x1275fb520 state=pending>,
 <Future at 0x1275fb5e0 state=pending>,
 <Future at 0x1275fb7f0 state=pending>,
 <Future at 0x1275fb8b0 state=pending>,
 <Future at 0x1275fbac0 state=pending>,
 <Future at 0x1275fbb80 state=pending>,
 <Future at 0x1275fbd90 state=pending>,
 <Future at 0x1275fbe50 state=pending>,
 <Future at 0x1275fb070 state=pending>,
 <Future at 0x1275fb100 state=pending>,
 <Future at 0x1275fb250 state=pending>,
 <Future at 0x1275fb2e0 state=pending>,
 <Future at 0x1276260d0 state=pending>,
 <Future at 0x1276261c0 state=pending>,
 <Future at 0x127626f40 state=pending>,


In [35]:
execution.progress()

{'ready': 60, 'active': 8, 'finished': 188, 'failed': 0}

In [36]:
results = campaign.analyse(qoi_cols=['distance'])

In [38]:
results.sobols_total()

{'distance': {'theta1': array([1.31166456e-14]),
  'theta2': array([0.35605895]),
  'theta3': array([0.44497095]),
  'theta4': array([0.35605895])}}